
# Проект‑практикум: Парсинг данных • Вариант для студентов

**Задача:** реализовать рабочий пайплайн парсинга (сбор → очистка → проверка) своими руками.  
**Правило:** в кодовых ячейках ниже оставлены только комментарии‑подсказки. Заменяйте их на **свой код**.

**Этапы**
1. Фаза 0 — подготовка окружения и структуры.
2. Фаза 1 — мини‑парсер «Quotes to Scrape».
3. Фаза 2 — маркетплейс (двухшаговый: категория → карточка).
4. Фаза 3 — очистка и валидация.
5. Фаза 4 — воспроизводимость (мини‑конвейер) и смоук‑тесты.

> Подсказка: используйте понятные имена переменных, функции с докстрингами и логирование.


## 1. **Фаза 0 — подготовка окружения и структуры**

Это как настройка рабочего стола перед началом проекта:

* мы устанавливаем все нужные программы и библиотеки (инструменты),
* создаём папки для хранения данных: `raw` (сырые), `processed` (очищенные), `logs` (логи).

👉 Цель: подготовить «чистое рабочее место», чтобы дальше всё было организованно.

---

### 2. **Фаза 1 — мини-парсер «Quotes to Scrape»**

Это тренировочный этап. Мы берём сайт для обучения (quotes.toscrape.com), где размещены цитаты.

* Скачиваем их при помощи кода на Python,
* сохраняем в таблицу (CSV) с колонками «цитата», «автор», «теги».

👉 Цель: научиться базовому парсингу и проверить, что связка `requests + BeautifulSoup + CSV` работает.

---

### 3. **Фаза 2 — маркетплейс (категория → карточка)**

Здесь уже настоящий сайт — маркетплейс (Lalafo).

* Находим страницу категории (например, электроника),
* собираем ссылки на карточки товаров,
* переходим в каждую карточку и вытаскиваем данные: название товара и цену.

👉 Цель: освоить более сложный сценарий «двухшагового» парсинга: сначала ссылки, потом сами объекты.

---

### 4. **Фаза 3 — очистка и валидация**

Сырые данные редко бывают аккуратными. Поэтому:

* удаляем дубликаты и пустые строки,
* приводим цены к числам и фильтруем нереалистичные значения,
* добавляем служебные поля (источник, дата сбора, категория).

👉 Цель: получить чистый датасет, готовый для анализа или загрузки в базу.

---

### 5. **Фаза 4 — воспроизводимость (пайплайн) и смоук-тесты**

Здесь мы собираем всё вместе:

* одна команда запускает весь процесс — парсинг + очистку,
* параметры (например, какая категория, сколько страниц) задаём через конфиг или `.env`,
* смоук-тесты проверяют, что данные корректны (есть все колонки, нет пустых URL, цены > 0).

👉 Цель: сделать проект удобным и повторяемым — так, чтобы его можно было запустить заново через неделю или дать другому человеку, и всё сработает.

---

## Фаза 0. Подготовка окружения

In [4]:
# TODO: Установите зависимости проекта через pip.
# Подсказка: используйте модуль запуска пакетов из Python или магию Jupyter (%pip).
# Обязательные пакеты: requests, beautifulsoup4, lxml, pandas, python-dotenv, tqdm (и при желании pyarrow).
# Пример шага: установить список пакетов и вывести подтверждение установки.

# Установка зависимостей
try:
    import sys, subprocess
    pkgs = ["requests", "beautifulsoup4", "lxml", "pandas", "python-dotenv", "tqdm", "pyarrow"]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
    print("✅ Зависимости установлены/обновлены")
except Exception as e:
    print("⚠️ Установите пакеты вручную:", e)

✅ Зависимости установлены/обновлены


In [8]:
# Хелперы для отображения и скачивания
import pandas as pd
from IPython.display import display
from pathlib import Path

IN_COLAB = False
try:
    import google.colab  # type: ignore
    from google.colab import data_table, files  # type: ignore
    data_table.enable_dataframe_formatter()
    IN_COLAB = True
except Exception:
    files = None  # type: ignore

def show_df(df: pd.DataFrame, title: str, n: int = 20):
    if title:
        print(title)
    display(df.head(n))

def show_csv(path: Path, title: str, n: int = 20):
    if not path.exists():
        print(f"Файл не найден: {path}")
        return
    try:
        df = pd.read_csv(path)
        show_df(df, title, n)
    except Exception as e:
        print("Не удалось прочитать CSV:", e)

def download_file(path: Path):
    """Скачивание файла в Colab. В Jupyter просто сообщает путь."""
    if not path.exists():
        print(f"Файл не найден: {path}")
        return
    if IN_COLAB and files is not None:
        files.download(str(path))
    else:
        print(f"Скачивание доступно в Colab. Путь к файлу: {path}")

In [9]:
# TODO: Создайте структуру папок проекта: data/raw, data/processed, logs.
# Подсказка: используйте pathlib.Path и метод mkdir(..., parents=True, exist_ok=True).
# Проверьте, что директории появились, выведите их абсолютные пути.
from pathlib import Path

BASE_DIR = Path.cwd()
RAW_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'
LOG_DIR = BASE_DIR / 'logs'

for path in [RAW_DIR, PROCESSED_DIR, LOG_DIR]:
  path.mkdir(parents = True, exist_ok = True)

print(f'Base dir: {BASE_DIR}\nRaw dir: {RAW_DIR}\nPocessed dir: {PROCESSED_DIR}\nLogs dir: {LOG_DIR}')

Base dir: D:\AIAcademy\Parser_1_project
Raw dir: D:\AIAcademy\Parser_1_project\data\raw
Pocessed dir: D:\AIAcademy\Parser_1_project\data\processed
Logs dir: D:\AIAcademy\Parser_1_project\logs


In [ ]:
# TODO: Задайте базовые параметры проекта (в виде переменных):
# - CATEGORY (строка пути категории маркетплейса)
# - PAGES_LIMIT (int, ограничение страниц)
# - PER_PAGE_LIMIT (int, ограничение карточек на страницу)
# - DELAY и ITEM_DELAY (float, задержки между запросами)
# - SOURCE и CURRENCY (строки)
# Выведите значения параметров для самопроверки.


### Утилиты: логирование

In [11]:
# TODO: Реализуйте функцию get_logger(name) для логирования в консоль и файл.
# Требования:
# - уровень INFO;
# - 2 хэндлера: StreamHandler (консоль) и FileHandler (в logs/<name>.log);
# - формат логов: "%(asctime)s - %(levelname)s - %(name)s - %(message)s".
# Подсказка: не дублируйте хэндлеры при повторных запусках в Jupyter.


import logging

def get_logger(name: str, log_dir: Path = LOG_DIR) -> logging.Logger:
    log_dir.mkdir(parents=True, exist_ok=True)
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    if logger.handlers:
        return logger
    ch = logging.StreamHandler(); ch.setLevel(logging.INFO)
    fh = logging.FileHandler(log_dir / f"{name}.log", encoding="utf-8"); fh.setLevel(logging.INFO)
    fmt = logging.Formatter("%(asctime)s - %(levelname)s - %(name)s - %(message)s")
    ch.setFormatter(fmt); fh.setFormatter(fmt)
    logger.addHandler(ch); logger.addHandler(fh)
    return logger


## Фаза 1. Мини‑парсер — Quotes to Scrape

In [12]:

# TODO: Импортируйте необходимые модули для сетевых запросов и HTML‑парсинга.
# Рекомендуемые инструменты: requests, bs4.BeautifulSoup, time.
# Создайте константы для базового URL и пути для CSV.


In [ ]:

# TODO: Реализуйте функцию fetch(url) с повторами и таймаутом.
# Логика:
# 1) выполнить GET с заголовком User-Agent;
# 2) при ошибке логировать предупреждение и делать экспоненциальную паузу;
# 3) после N попыток выбросить исключение.


In [ ]:

# TODO: Реализуйте функцию parse_quotes_page(html), которая:
# - находит блоки цитат;
# - извлекает поля: text, author, tags (теги объединить символом '|');
# - находит ссылку на следующую страницу (или None).
# Верните кортеж (список_словарей, next_url).


In [ ]:

# TODO: Реализуйте функцию save_csv(rows, path, fieldnames, write_header=False).
# Требования:
# - создать директорию при необходимости;
# - использовать csv.DictWriter;
# - корректная работа с UTF-8 и переносами строк.


In [ ]:

# TODO: Реализуйте функцию run_quotes_scraper(), которая:
# - инициализирует логгер;
# - удаляет существующий CSV (если он есть);
# - итерируется по страницам, пока есть next_url;
# - сохраняет данные постранично;
# - возвращает общее количество собранных цитат.
# В конце выведите краткую статистику.


## Фаза 2. Маркетплейс — двухшаговый парсинг

In [ ]:

# TODO: Подготовьте импорты и переменные для маркетплейса:
# - BASE_URL;
# - MARKET_OUT (путь к RAW CSV);
# - логгер;
# Подсказка: используйте те же утилиты логирования.


In [ ]:

# TODO: Реализуйте функцию fetch_market(url) по аналогии с fetch(url), но с другими таймаутами/повторами.
# Требования такие же: заголовок User-Agent, обработка исключений, паузы между попытками.


In [ ]:

# TODO: Реализуйте функцию get_item_links_from_category(html):
# - распарсьте HTML категории;
# - соберите все <a href="...">, нормализуйте в абсолютные ссылки;
# - отфильтруйте только ссылки, у которых в пути встречается '/ads/';
# - удалите дубликаты, сохранив порядок.
# Верните список ссылок на объявления.


In [ ]:

# TODO: Реализуйте функцию parse_item_page(html):
# - извлеките заголовок из <h1>;
# - из всего текста страницы извлеките первую цену перед 'KGS' (регулярное выражение);
# - верните словарь {'title': ..., 'price': ...}.


In [ ]:

# TODO: Реализуйте функцию get_next_page_url(html):
# - найдите ссылку на следующую страницу (rel='next' или другой селектор);
# - верните абсолютный URL или None.


In [ ]:

# TODO: Реализуйте функцию run_market_scraper(...), которая:
# - принимает category, pages_limit, per_page_limit, задержки;
# - собирает ссылки объявлений с каждой страницы категории;
# - для каждой ссылки загружает карточку и извлекает title/price;
# - постранично сохраняет результат в CSV;
# - возвращает общее количество валидных объявлений.
# Выведите статистику по страницам и итог.


## Фаза 3. Очистка и валидация

In [ ]:

# TODO: Импортируйте pandas и другие нужные модули (datetime, re).
# Объявите пути: RAW_CSV (в data/raw), CLEAN_CSV и CLEAN_PQ (в data/processed).


In [ ]:

# TODO: Напишите функцию normalize_title(s), которая:
# - обрезает пробелы по краям;
# - заменяет множественные пробелы на один.
# Верните нормализованную строку.


In [ ]:

# TODO: Реализуйте функцию clean_market_data(...):
# - загрузите RAW CSV;
# - проверьте наличие обязательных столбцов (title, price, url);
# - удалите дубликаты по url и пустые url;
# - приведите price к числу (coerce), оставьте только > 0 и разумный верхний порог;
# - добавьте служебные поля: source, category, currency, date_scraped (ISO‑8601 UTC);
# - сохраните CSV и Parquet (если установлен соответствующий движок);
# - верните количество валидных строк.
# Выведите предпросмотр очищенных данных.


## Фаза 4. Воспроизводимость: пайплайн и смоук‑тесты

In [ ]:

# TODO: Реализуйте функцию run_pipeline(), которая:
# - запускает мини‑парсер (опционально);
# - запускает маркет‑парсер;
# - запускает очистку;
# - печатает сводку scraped/cleaned и возвращает кортеж.


In [ ]:

# TODO: Реализуйте функцию smoke_tests(), которая проверяет базовые инварианты:
# - наличие RAW и CLEAN файлов;
# - clean не длиннее raw;
# - наличие столбцов в clean: title, price, currency, url, source, category, date_scraped;
# - все цены в clean > 0;
# - нет пустых url.
# В конце выведите 'SMOKE CHECKS OK' при успехе.



### Этика и право
- Соблюдайте robots.txt и условия сайта.
- Держите паузы между запросами, не создавайте лишней нагрузки.
- Не собирайте персональные данные, не обходите защиту.

### DoD — критерии приёмки
- Мини‑парсер собрал ≥ 100 цитат;
- Маркет‑парсер собрал ≥ 200 объявлений при разумных лимитах;
- Очищенный CSV/Parquet содержит нужные столбцы;
- Смоук‑тесты проходят без ошибок;
- Весь конвейер запускается одной функцией.
